# Cleaning

In [67]:
import chardet
import pandas as pd
import datetime

In [68]:
#with open('data/owner.csv', 'rb') as f:
#    result = chardet.detect(f.read())  # or readline if the file is large
#
#print(result)

In [69]:
# display full url for user_profile for instance
#pd.set_option('display.max_colwidth', 1000)

In [70]:
owner = pd.read_csv("data/owner.csv",sep=';', encoding='ISO-8859-1')
prop = pd.read_csv("data/property.csv", sep=';', encoding='ISO-8859-1')

In [6]:
print("shape of owner is :" + str(owner.shape))
print("shape of prop is :" + str(prop.shape))

shape of owner is :(15127, 7)
shape of prop is :(18686, 14)


In [7]:
# vérification des années
#owner["Année de Birthdate"].unique()

In [8]:
# make a dictionary that maps the month name in french to a number
french_to_eng = {'janvier':'1','février':'2','mars': '3','avril': '4','mai':'5','juin':'6','juillet': '7','août': '8','septembre': '9','octobre':'10','novembre':'11','décembre':'12'}

# make new columsn for day month and year. For month, map the french name to month numbers
owner['day_sign_up'] = owner['Jour de Created At User'].apply(lambda x : x.split(' ')[0])
owner['month_sign_up'] = owner['Jour de Created At User'].apply(lambda x : x.split(' ')[1]).map(french_to_eng)
owner['year_sign_up'] = owner['Jour de Created At User'].apply(lambda x : x.split(' ')[2])

# make date time column from year, month and day.
owner['user_sign_up_date'] = pd.to_datetime(owner['year_sign_up']+'-'+owner['month_sign_up']+'-'+owner['day_sign_up'],format='%Y-%m-%d', errors='ignore')

owner = owner.drop(columns=['day_sign_up', 'month_sign_up', 'year_sign_up'])

In [9]:
owner['Jour de Last Login Date'].fillna(owner['Jour de Created At User'], inplace=True)

In [10]:
# make new columsn for day month and year. For month, map the french name to month numbers
owner['day_sign_in'] = owner['Jour de Last Login Date'].apply(lambda x : x.split(' ')[0])
owner['month_sign_in'] = owner['Jour de Last Login Date'].apply(lambda x : x.split(' ')[1]).map(french_to_eng)
owner['year_sign_in'] = owner['Jour de Last Login Date'].apply(lambda x : x.split(' ')[2])

# make date time column from year, month and day.
owner['user_sign_in_date'] = pd.to_datetime(owner['year_sign_in']+'-'+owner['month_sign_in']+'-'+owner['day_sign_in'],format='%Y-%m-%d', errors='ignore')

owner = owner.drop(columns=['day_sign_in', 'month_sign_in', 'year_sign_in'])

In [11]:
owner['user_sign_in_date'] = pd.to_datetime(owner['user_sign_in_date'])
owner['user_sign_up_date'] = pd.to_datetime(owner['user_sign_up_date'])

In [12]:
owner['today_date']= datetime.date.today()
owner['today_date']= pd.to_datetime(owner.today_date)

In [13]:
owner['seniority'] = owner.today_date - owner.user_sign_in_date
owner['days_since_last_login'] = owner.today_date - owner.user_sign_up_date

# quand owner['days_since_last_login'] vaut 0, on émet l'hypothèse que le compte est inactif

In [14]:
owner['year'] = datetime.date.today().year
owner['Année de Birthdate'] = owner['Année de Birthdate'].fillna(datetime.date.today().year)
owner['age'] = owner.year - owner['Année de Birthdate']
owner = owner.drop(columns=['Année de Birthdate', 'year', 'Jour de Created At User', 'Jour de Last Login Date'])

In [15]:
owner.head()

,Owner ID,Gender,Languages,Country,user_sign_up_date,user_sign_in_date,today_date,seniority,days_since_last_login,age
0,0a0e4-ad50-4b3a-81fd-a29ed0000,male,[],NaN,2018-05-12,2019-02-19,2019-06-15,116 days,399 days,0.0
1,0a1e1-1cfe-46d0-9069-e567bc9c3,female,[],NaN,2018-06-11,2018-06-26,2019-06-15,354 days,369 days,0.0
2,0a2b8-f8c2-4a2c-b6f8-d2f901abe,female,[],NaN,2018-04-17,2018-04-17,2019-06-15,424 days,424 days,0.0
3,0a2bd-8842-4067-b52e-0a8922be2,female,"[""en"", ""fr""]",NaN,2017-06-26,2017-07-17,2019-06-15,698 days,719 days,0.0
4,0a3ef-f75e-463c-b52b-fa39184d5,male,[],NaN,2018-03-09,2018-03-09,2019-06-15,463 days,463 days,0.0


In [16]:
# Je ne traite pas les 28 lignes présentant des erreurs de formatages
df_owner = owner[owner["Languages"].str.startswith('{') == False]
df_owner = df_owner.rename(columns={"owner_id":"id_owner","gender":"gaaender","languages":"languages","country":"country","user_sign_up_date":"user_sign_up_date","user_sign_in_date":"user_sign_in_date","today_date":"today_date","seniority":"seniority","days_since_last_login":"days_since_last_login","age": "age"})
df_owner.columns = map(str.lower, df_owner.columns)
df_owner.columns = df_owner.columns.str.replace(' ', '_')
print(df_owner.shape)
print(owner.shape)
print(owner[owner["Languages"].str.startswith('{')].shape)

(15099, 10)
(15127, 10)
(28, 10)


In [17]:
# Boolean values
funrished_mapping= {'FAUX':False, 'VRAI':True}
prop.Furnished = prop.Furnished.map(funrished_mapping)
prop.Furnished.unique()

array([nan, False, True], dtype=object)

In [18]:
# make a dictionary that maps the month name in french to a number
french_to_eng = {'janv':'1','févr':'2','mars': '3','avr': '4','mai':'5','juin':'6','juil': '7','août': '8','sept': '9','oct':'10','nov':'11','déc':'12'}
year_mapping = {'13':'2013','14':'2014','15':'2015','16':'2016','17':'2017','18':'2018', '19':'2019'}

# make new columsn for day month and year. For month, map the french name to month numbers
prop['day_offline_at'] = prop[prop['Jour de Offline At'].notnull() == True]['Jour de Offline At'].apply(lambda x : x.split('-')[0])
prop['month_offline_at'] = prop[prop['Jour de Offline At'].notnull() == True]['Jour de Offline At'].apply(lambda x : x.split('-')[1]).map(french_to_eng)
prop['year_offline_at'] = prop[prop['Jour de Offline At'].notnull() == True]['Jour de Offline At'].apply(lambda x : x.split('-')[2]).map(year_mapping)

# make date time column from year, month and day.
prop['offline_date'] = pd.to_datetime(prop['year_offline_at']+'-'+prop['month_offline_at']+'-'+prop['day_offline_at'],format='%Y-%m-%d', errors='ignore')

prop = prop.drop(columns=['day_offline_at', 'month_offline_at', 'year_offline_at'])

In [19]:
# make a dictionary that maps the month name in french to a number

# make new columsn for day month and year. For month, map the french name to month numbers
prop['day_online_at'] = prop[prop['Jour de Online At'].notnull() == True]['Jour de Online At'].apply(lambda x : x.split('-')[0])
prop['month_online_at'] = prop[prop['Jour de Online At'].notnull() == True]['Jour de Online At'].apply(lambda x : x.split('-')[1]).map(french_to_eng)
prop['year_online_at'] = prop[prop['Jour de Online At'].notnull() == True]['Jour de Online At'].apply(lambda x : x.split('-')[2]).map(year_mapping)

# make date time column from year, month and day.
prop['online_date'] = pd.to_datetime(prop['year_online_at']+'-'+prop['month_online_at']+'-'+prop['day_online_at'],format='%Y-%m-%d', errors='ignore')

prop = prop.drop(columns=['day_online_at', 'month_online_at', 'year_online_at'])

In [20]:
# make a dictionary that maps the month name in french to a number

# make new columsn for day month and year. For month, map the french name to month numbers
prop['day_created_at'] = prop[prop['Jour de Created At (Properties)'].notnull() == True]['Jour de Created At (Properties)'].apply(lambda x : x.split('-')[0])
prop['month_created_at'] = prop[prop['Jour de Created At (Properties)'].notnull() == True]['Jour de Created At (Properties)'].apply(lambda x : x.split('-')[1]).map(french_to_eng)
prop['year_created_at'] = prop[prop['Jour de Created At (Properties)'].notnull() == True]['Jour de Created At (Properties)'].apply(lambda x : x.split('-')[2]).map(year_mapping)

# make date time column from year, month and day.
prop['proprety_creation_date'] = pd.to_datetime(prop['year_created_at']+'-'+prop['month_created_at']+'-'+prop['day_created_at'],format='%Y-%m-%d', errors='ignore')

prop = prop.drop(columns=['day_created_at', 'month_created_at', 'year_created_at','Jour de Offline At','Jour de Online At','Jour de Created At (Properties)'])

In [21]:
# Delete first row
df_prop = prop[1:-1]

In [22]:
df_prop.columns = map(str.lower, df_prop.columns)
df_prop.columns = df_prop.columns.str.rstrip()
df_prop.columns = df_prop.columns.str.replace(' ', '_')

In [23]:
df_owner.head()

,owner_id,gender,languages,country,user_sign_up_date,user_sign_in_date,today_date,seniority,days_since_last_login,age
0,0a0e4-ad50-4b3a-81fd-a29ed0000,male,[],NaN,2018-05-12,2019-02-19,2019-06-15,116 days,399 days,0.0
1,0a1e1-1cfe-46d0-9069-e567bc9c3,female,[],NaN,2018-06-11,2018-06-26,2019-06-15,354 days,369 days,0.0
2,0a2b8-f8c2-4a2c-b6f8-d2f901abe,female,[],NaN,2018-04-17,2018-04-17,2019-06-15,424 days,424 days,0.0
3,0a2bd-8842-4067-b52e-0a8922be2,female,"[""en"", ""fr""]",NaN,2017-06-26,2017-07-17,2019-06-15,698 days,719 days,0.0
4,0a3ef-f75e-463c-b52b-fa39184d5,male,[],NaN,2018-03-09,2018-03-09,2019-06-15,463 days,463 days,0.0


# Analysis

In [52]:
df_owner.groupby('gender').count().owner_id

gender
female    7495
male      7604
Name: owner_id, dtype: int64

In [35]:
import plotly.plotly as py
import plotly.graph_objs as go

In [152]:
trace = go.Scatter(
    x = df_owner[['owner_id','days_since_last_login']].owner_id.tolist(),
    y = df_owner['days_since_last_login'].apply(lambda x : x.days).tolist(),
    mode = 'markers'
)

data = [trace]
py.iplot(data, filename='basic-scatter')

On distingue deux groupes d'utilisateurs: les actifs ayant réaliser une connexion au cours des 500 derniers jours, et ceux qui semblent avoir quitter la plateforme il y a envion 700 jours.

In [169]:
df_owner.dtypes

owner_id                          object
gender                            object
languages                         object
country                           object
user_sign_up_date         datetime64[ns]
user_sign_in_date         datetime64[ns]
today_date                datetime64[ns]
seniority                timedelta64[ns]
days_since_last_login    timedelta64[ns]
age                              float64
dtype: object

In [181]:
df_owner[df_owner.age <= -1]

,owner_id,gender,languages,country,user_sign_up_date,user_sign_in_date,today_date,seniority,days_since_last_login,age
5547,90b16-913d-4804-867d-c7d7c7b33,female,"[""de"", ""en"", ""fr""]",NaN,2015-12-21,2019-06-08,2019-06-15,7 days,1272 days,-11.0
12585,d62f8-0adf-4935-84ca-ed686825e,male,"[""fr"", ""en""]",NaN,2017-02-05,2018-03-05,2019-06-15,467 days,860 days,-952.0


In [175]:
-> traduire les timedelta en int
df_owner.describe()

,seniority,days_since_last_login,age
count,15099,15099,15099.000000
mean,264 days 02:43:56.518974,395 days 14:47:13.936022,5.893172
std,196 days 14:13:31.322066,249 days 20:32:28.551864,18.860931
min,4 days 00:00:00,4 days 00:00:00,-952.000000
25%,82 days 00:00:00,255 days 00:00:00,0.000000
50%,295 days 00:00:00,374 days 00:00:00,0.000000
75%,379 days 00:00:00,448 days 00:00:00,0.000000
max,1272 days 00:00:00,1272 days 00:00:00,119.000000
